# CS295/395: Secure Distributed Computation
## In-Class Exercise, Week of 9/5/2022

In [1]:
import galois

GF = galois.GF((2**7)-1)

In [10]:
# this is a field element from the field GF(127)
GF(5)

assert GF(100) + GF(100) == GF(73)

## Question 1

Implement the functions `plusFE` and `multFE` using the `galois` library.

In [11]:
# YOUR CODE HERE
def plusFE(a: galois.GF, b: galois.GF) -> galois.GF:
    return a+b

def multFE(a: galois.GF, b: galois.GF) -> galois.GF:
    return a*b

In [12]:
a = GF(100)
b = GF(70)

assert plusFE(a, b) == GF(43)
assert plusFE(a, b) == 43
assert multFE(a, b) == GF(15)
assert multFE(a, b) == 15

Consider the following functionality and protocol for realizing it.

\begin{equation*}
\textbf{Functionality: Summation}\\
\fbox{$\mathcal{F}(x_1, \dots, x_n) = x_1 + \dots + x_n$}
\end{equation*}


**Protocol: Secure Summation with Additive Secret Sharing**
- **Round 1**: Each party $P_i$ sends one share of its input $x_i$ to each other party, keeping one share for itself.
- **Round 2**: Each party $P_i$ sums the shares it holds (including both the shares it has received and the share it kept for itself). Each party sends its sum to all other parties.
- **Opening**: Each party adds up the sums it receives and the sum it computed to obtain the final output.

## Question 2

Does the protocol above securely realize the functionality in the presence of a *semi-honest* adversary? If not, why not?

Yes, it does.
- The protocol gives no more information to the adversary than the functionality
- The functionality might reveal someone's secret input ig the adversary controls all the parties except one, but this is outside our secrity definition
- If more than one party is honest, no honest party's secret input is reveal by the functionality
- BECAUSE if you controlled everyone and had the functionality but DIDNT even run the protocol they can still reveal the information

## Question 3

Does the protocol above securely realize the functionality in the presence of a *malicious* adversary? If not, why not?

No, a malicious adversary could add different values in round 2 than it recieves and all tha parties would get the wrong answer.

## Question 4

Consider a semi-honest adversary. In the real world, what information is included in the view of the (potentially corrupted) party $p_i$?

Generally a view includes a parties private input and all messages recieved by the party. In this case:
- the input Xi
- the shares of each parties inputs
- the shares of the final sum.

## Question 5

Describe a *simulator* for the ideal world, that constructs views which are indistinguishable from the ones described in Question 1.

For party i we have
- Private input x_i
- final output of ideal func (sum of all xs)

We need to construct
- the input x_i (We have this explicitly from ideal func)
- shares of each other party's input (pick a random number because additive secret shares look like random numbers)
- shares of the final sum (we have this as output from ideal func)

Because we cannot distinguish between real random numbers and the additive secret random numbers, we cannot distinguish from real life vs protocol and we are secure.

In [15]:
from collections import defaultdict
class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self):
        """Initialize the dictionary to hold received messages."""
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

## Question 6

Implement the `InsecureAggregationParty` class, which adds up the parties' inputs *insecurely*.

In [16]:
class InsecureAggregationParty(Party):
    def round1(self, parties, input):
        self.input = GF(input)

        for party in parties:
            self.send(party, 1, self.input)

    def round2(self):
        self.output = GF(self.received[1]).sum()

    def get_view(self):
        return (self.input, self.output, self.received[1])

In [17]:
# TEST CASE for question 1

# 5 parties
parties = [InsecureAggregationParty() for _ in range(5)]

# run round 1
for party in parties:
    party.round1(parties, 5)

# run round 2 and output
for party in parties:
    party.round2()
    #print('Party output:', party.output, type(party.output))
    assert party.output == GF(25)

## Question 7

Implement a simulator for the `InsecureAggregationParty` protocol, which constructs the view for party $i$ indistinguishable from the real-world view of party $i$ in the protocol, using only the inputs and output of the ideal functionality.

In [ ]:
def simulator(n, i, inputs, output):
    """Simulates a real-world view in the ideal world. Outputs a 
    3-tuple: (input, output, received messages from round 1)"""
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# TEST CASE for question 2

# the real-world view of each party should be indistinguishable
# from the simulated view in the ideal world
# there is no randomness here, so they are really equal!
all_inputs = [5 for _ in range(5)]
for i, party in enumerate(parties):
    assert party.get_view() == simulator(5, i, all_inputs, 25)

## Question 8

Can you construct a simulator for `InsecureAggregationParty` without using the input for all $n$ parties? If not, why not? What does this mean about the security of `InsecureAggregationParty`?

YOUR ANSWER HERE